In [ ]:
import pandas as pd
import numpy as np
from gensim.corpora import Dictionary

df = pd.read_csv('data2.txt', delimiter = '\t', names = ('keyword', 'sentence'))

keywords = [k.split(' ') for k in df['keyword'].values]
sentences = [s.split(' ') for s in df['sentence'].values]

dic = Dictionary(keywords + sentences)

q_maxlen = np.max([len(q) for q in keywords])
a_maxlen = np.max([len(a) for a in sentences])

In [ ]:
from keras.layers import Input, Dense, Flatten, Dropout, GRU, Activation, concatenate
from keras.models import Sequential, Model
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import RepeatVector
from keras.layers.wrappers import TimeDistributed

def discriminator(input_shape1, input_shape2, n_units):
    model = Sequential()

    lrelu = LeakyReLU()
    
    input1 = Input(shape = input_shape1)
    input2 = Input(shape = input_shape2)

    x = concatenate([input1, input2], axis = 1)

    x = GRU(n_units[0])(x)
    x = Dropout(0.2)(x)
    x = Dense(n_units[1], activation = lrelu)(x)

    output = Dense(1, activation = 'sigmoid')(x)
    
    model = Model([input1, input2], output)
    
    model.summary()
    
    return model

def generator(input_shape, output_shape, n_units):
    model = Sequential()

    model.add(GRU(n_units[0], input_shape = input_shape))

    model.add(RepeatVector(output_shape[0]))
    model.add(Dropout(0.2))
    model.add(GRU(n_units[1], return_sequences = True))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(output_shape[1], activation = 'softmax')))

    model.summary()

    input = Input(shape = input_shape)

    return Model(input, model(input))


In [ ]:
from keras.optimizers import Adam

q_shape = (q_maxlen, len(dic))
a_shape = (a_maxlen, len(dic))

dis_opt = Adam(lr = 1e-5, beta_1 = 0.1)

dis = discriminator(q_shape, a_shape, [256, 128])
dis.compile(loss = 'binary_crossentropy', optimizer = dis_opt, metrics = ['acc'])

dis.trainable = False

gen = generator(q_shape, a_shape, [256, 512])
gen.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

x = Input(shape = q_shape)
y = dis([x, gen(x)])

m_opt = Adam(lr = 2e-4, beta_1 = 0.5)

model = Model(x, y)
model.compile(loss = 'binary_crossentropy', optimizer = m_opt)

In [ ]:
padding_one_hot = lambda ws, size: np.vstack((
    np.eye(len(dic))[dic.doc2idx(ws)],
    np.zeros((size - len(ws), len(dic)))
))

def random_data(batch_size):
    idx = np.random.randint(0, len(keywords), batch_size)
    return get_data(idx)

def get_data(idx):
    q = np.array([padding_one_hot(ws, q_maxlen) for ws in np.array(keywords)[idx]])
    a = np.array([padding_one_hot(ss, a_maxlen) for ss in np.array(sentences)[idx]])
    
    return (q, a)

def train(epochs, batch_size, gen_steps = 1, dis_steps = 1, con_steps = 1):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for ep in range(epochs):
        for i in range(gen_steps):
            perm = np.random.permutation(len(keywords))

            losses = []
            
            for j in range(0, len(keywords), batch_size):
                q, a = get_data(perm[i:i + batch_size])
                losses.append( gen.train_on_batch(q, a) )

            loss, acc = np.mean(losses, axis = 0)
            
            print(f'epoch = {ep}, step = gen-{i}, loss = {loss}, acc = {acc}')
            
        for i in range(dis_steps):
            q, a = random_data(batch_size)

            gen_a = gen.predict(q)
        
            loss_valid = dis.train_on_batch([q, a], valid)
            loss_fake = dis.train_on_batch([q, gen_a], fake)

            loss, acc = 0.5 * np.add(loss_valid, loss_fake)

            print(f'epoch = {ep}, step = dis-{i}, loss = {loss}, acc = {acc}, fake acc = {loss_fake[1]}')
            
        for i in range(con_steps):
            q, _ = random_data(batch_size)

            loss = model.train_on_batch(q, valid)

            print(f'epoch = {ep}, step = con-{i}, loss = {loss}')


In [ ]:
train(30, 100, 10)
train(100, 100, 5)

In [ ]:
def answer(kw):
    x = np.array([padding_one_hot(kw, q_maxlen)])

    y = gen.predict(x)

    res = [dic[np.argmax(w)] for w in y[0]]
    
    return ''.join(res)


In [ ]:
for i in range(10):
    q = ''.join(keywords[i])
    a = ''.join(sentences[i])

    r = answer(keywords[i])
    
    print(f'q = {q}, a = {a}, result = {r}' )
    print('-----')

